In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: Averaging history and use FFN

Here, we average the full history of a path and concatenate it to the current embedding (the total number of features that are passed into the FFN is 2 * sbert_embeddings.shape[0]).

Here, we will run the hyperparameter search to implement the FFN with the same parameters as the standard FFN baseline on the sentence embeddings.

In [7]:
num_epochs = 100
batch = 64
hidden_dim_sizes = [[64,64], [128,128], [256,256]]
dropout_rates =  [0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

In [8]:
ffn_mean_history, best_ffn_mean_history, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    path_indices=None,
    #data_split_seed=0,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/3 [00:00<?, ?it/s]



100%|██████████| 3/3 [00:50<00:00, 16.97s/it]




 33%|███▎      | 1/3 [01:40<03:21, 100.52s/it]



100%|██████████| 3/3 [00:43<00:00, 14.43s/it]




 67%|██████▋   | 2/3 [03:08<01:33, 93.31s/it] 



100%|██████████| 3/3 [00:59<00:00, 19.88s/it]




100%|██████████| 3/3 [05:06<00:00, 102.16s/it]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


100%|██████████| 5568/5568 [00:10<00:00, 548.25it/s]


In [9]:
ffn_mean_history

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,n_splits,batch_size,model_id,input_dim
0,focal,0.524309,0.523970,"[0.5112634671890304, 0.5366759517177344]",0.526916,"[0.5529661016949152, 0.5008665511265165]",0.526705,"[0.47540983606557374, 0.578]",0.253740,0.669039,...,"(64, 64)",0.2,0.0010,0,2,False,None,64,0.00,768
0,focal,0.518589,0.518477,"[0.5111326234269118, 0.5258215962441315]",0.520515,"[0.5454545454545454, 0.49557522123893805]",0.520437,"[0.4808743169398907, 0.56]",0.247394,0.676157,...,"(64, 64)",0.2,0.0010,1,2,False,None,64,0.00,768
0,focal,0.506196,0.495175,"[0.569767441860465, 0.4205816554809843]",0.500411,"[0.5236641221374045, 0.47715736040609136]",0.500386,"[0.6247723132969034, 0.376]",0.247915,0.672598,...,"(64, 64)",0.2,0.0010,12,2,False,None,64,0.00,768
0,focal,0.527169,0.526700,"[0.5118110236220472, 0.5415896487985212]",0.530091,"[0.556745182012848, 0.5034364261168385]",0.529794,"[0.47358834244080145, 0.586]",0.240271,0.658363,...,"(64, 64)",0.2,0.0010,123,2,False,None,64,0.00,768
0,focal,0.453765,0.435639,"[0.5367825383993532, 0.33449477351916374]",0.440725,"[0.48255813953488375, 0.3988919667590028]",0.446368,"[0.604735883424408, 0.288]",0.281400,0.679715,...,"(64, 64)",0.2,0.0010,1234,2,False,None,64,0.00,768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.495710,0.492136,"[0.534740545294635, 0.4495317377731529]",0.492777,"[0.5170068027210885, 0.4685466377440347]",0.492867,"[0.5537340619307832, 0.432]",0.272760,0.669039,...,"(256, 256)",0.1,0.0005,0,2,False,None,64,0.17,768
0,focal,0.503336,0.502751,"[0.4856860809476801, 0.519815668202765]",0.506112,"[0.5301724137931034, 0.48205128205128206]",0.506044,"[0.44808743169398907, 0.564]",0.259724,0.661922,...,"(256, 256)",0.1,0.0005,1,2,False,None,64,0.17,768
0,focal,0.528122,0.527875,"[0.5386766076421249, 0.5170731707317073]",0.528144,"[0.5515267175572519, 0.5047619047619047]",0.528206,"[0.5264116575591985, 0.53]",0.240372,0.679715,...,"(256, 256)",0.1,0.0005,12,2,False,None,64,0.17,768
0,focal,0.523356,0.511264,"[0.5881383855024711, 0.4343891402714932]",0.518421,"[0.5368421052631579, 0.5]",0.517137,"[0.6502732240437158, 0.384]",0.233040,0.701068,...,"(256, 256)",0.1,0.0005,123,2,False,None,64,0.17,768


In [10]:
best_ffn_mean_history

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,valid_recall_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,n_splits,batch_size,input_dim
0,focal,0.505243,0.505241,"[0.504297994269341, 0.5061845861084681]",0.506440,"[0.5301204819277109, 0.4827586206896552]",0.506437,"[0.4808743169398907, 0.532]",0.250549,0.676157,...,"[0.7653631284916201, 0.5196078431372549]","(64, 64)",0.2,0.0001,0,2,False,None,64,768
0,focal,0.518589,0.518337,"[0.5073170731707317, 0.5293569431500466]",0.520928,"[0.5462184873949579, 0.4956369982547993]",0.520794,"[0.47358834244080145, 0.568]",0.252958,0.665480,...,"[0.7318435754189944, 0.5490196078431373]","(64, 64)",0.2,0.0001,1,2,False,None,64,768
0,focal,0.520496,0.520489,"[0.5223171889838555, 0.5186602870813397]",0.521441,"[0.5456349206349206, 0.4972477064220184]",0.521455,"[0.5009107468123861, 0.542]",0.262187,0.683274,...,"[0.770949720670391, 0.5294117647058824]","(64, 64)",0.2,0.0001,12,2,False,None,64,768
0,focal,0.523356,0.523334,"[0.5201535508637236, 0.5265151515151515]",0.524848,"[0.5496957403651116, 0.5]",0.524812,"[0.4936247723132969, 0.556]",0.272823,0.686833,...,"[0.776536312849162, 0.5294117647058824]","(64, 64)",0.2,0.0001,123,2,False,None,64,768
0,focal,0.510010,0.509998,"[0.5076628352490421, 0.5123339658444023]",0.511359,"[0.5353535353535354, 0.48736462093862815]",0.511348,"[0.48269581056466304, 0.54]",0.255072,0.665480,...,"[0.7430167597765364, 0.5294117647058824]","(64, 64)",0.2,0.0001,1234,2,False,None,64,768


In [11]:
best_ffn_mean_history["f1"].mean()

0.5154799576236101

In [12]:
best_ffn_mean_history["precision"].mean()

0.5170031111981337

In [13]:
best_ffn_mean_history["recall"].mean()

0.5169693989071039

In [14]:
np.stack(best_ffn_mean_history["f1_scores"]).mean(axis=0)

array([0.51234973, 0.51861019])

In [15]:
np.stack(best_ffn_mean_history["precision_scores"]).mean(axis=0)

array([0.54140463, 0.49260159])

In [16]:
np.stack(best_ffn_mean_history["recall_scores"]).mean(axis=0)

array([0.4863388, 0.5476   ])